In [19]:
from langchain.chat_models import BedrockChat
from langchain.schema import HumanMessage
from langchain.agents import load_tools
import os
import boto3

# os.environ['AWS_ACCESS_KEY_ID'] = ""
# os.environ['AWS_SECRET_ACCESS_KEY'] = ""

BEDROCK_CLIENT = boto3.client("bedrock-runtime", 'us-east-1')

llm = BedrockChat(model_id="anthropic.claude-v2:1", model_kwargs={"temperature": 0.1}, client=BEDROCK_CLIENT)

### LangSmith Tracing

In [20]:
import os
# from uuid import uuid4

# unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__5512334fadb54637a91e0b644e843283"  # Update to your API key

### 함수 정의

In [21]:
import json
import requests
import pandas as pd

fitcloud_url = "https://aws-dev.fitcloud.co.kr"
corpId = "KDjAqAG0TnEAAAFK5eqDUL0A"

# account 별 월 별 사용량 (saving plan 포함)
# type: usage - ApplySavingsPlanCompute
def corp_month(
    start_month: str, 
    end_month: str,
    token: str,
    groupBy="account",
    ):
  api_url = fitcloud_url + "/service/trend/corp/month"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": start_month,
      "to": end_month,
      "groupBy": groupBy,
  }

  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # 일반 월 사용량에서 SavingPlan 가격을 제함
    return pd.DataFrame(resp.json())

  else:
    print("error")
#------------------------------------------------------------------------------------
# 월 입력값이  from: '201901', to: '202210'형태일 경우
# 시작 월부터 종료 월까지 리스트로 출력
from datetime import datetime, timedelta

def month_range(start_month, end_month):
    # Create datetime objects for start and end dates
    start_date = datetime.strptime(start_month, "%Y%m")
    end_date = datetime.strptime(end_month, "%Y%m")

    # Initialize list to store months
    months_list = []

    # Iterate over months and add them to the list
    current_month = start_date
    while current_month <= end_date:
        months_list.append(current_month.strftime("%Y%m"))
        current_month = (current_month + timedelta(days=32)).replace(day=1)

    return months_list
#-------------------------------------------------------------------------------------
# account 일자별 사용량을 반환
def ondemand_account_day(
    accountId: str, 
    day_from: str, 
    day_to: str, 
    token: str) -> float:
  api_url = fitcloud_url + "/ondemand/account/day"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": day_from,
      "to": day_to,
      "accountId": accountId,
  }
  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # JSON 형식으로 응답을 파싱 후 usageFee 합계를 구하기 위해 dataframe 의 변환
    df = pd.DataFrame(resp.json())
    usage_sum = round( df['usage_fee'].astype("Float32").sum(), 2)
    return usage_sum

  else:
    print("error")

def corp_month_internal(start_month: str, end_month: str, accountId: str, token: str):
  """calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
  """
  json_data = corp_month(start_month, end_month, token)
  df = pd.DataFrame(json_data)
  # accountId = accountId
  # account에 관련된 데이터 추출
  # df = df.query("accountId==@accountId")
  df = df[df['accountId'] == accountId]
  # 기간 내 월 리스트 추출
  month_list = month_range(start_month, end_month)
  # 월 column의 data type을 numeric으로 변환
  df_acc = df.copy()
  df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
  # 내부 사용자용 filter: 합산에 포함시킬 항목
  internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
  # internal_filter = ['Usage','ApplySavingsPlanCompute']
  df_int = df_acc.query("type in @internal_filter")
  sum = df_int[month_list].sum().sum()
  return sum

   

### 툴 정의

In [22]:
# StructuredTool dataclass
# https://blog.langchain.dev/structured-tools/

from langchain.tools.base import StructuredTool

usage = StructuredTool.from_function(
    func=corp_month_internal,
    name="get_usage",
    description="calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars."
)

tools = [ usage ]

### Prompt

In [23]:
prompt_template = """
You are a helfull assistant, your task is help the user achieve is goal.
In this environment you have access to a set of tools you can use to answer the user's question.
Use them in a smart way to fullfill our objetives. Always consider the history of previous steps to ensure a continuos progress towards the objetive.
start_month and end_month string format must be in the form '%Y%m'. for example '202309'.
When you have the answer for the user, always answer it in this forma bellow:
Final Answer: answer to user


You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>


User:
{input}

"""

### Format Tool

In [24]:
from langchain_core.tools import BaseTool

from langchain_community.utils.openai_functions import (
    FunctionDescription,
    ToolDescription,
    convert_pydantic_to_openai_function,
)

def format_tool_to_anthropic_function(tool: BaseTool) -> FunctionDescription:
	"""Format tool into the Anthropic function API."""
	new_tool = f"""<tool_description>
<tool_name>{tool.name}</tool_name>
<description>
{tool.description}
<parameters>
</parameters>
</tool_description>"""
	return new_tool

### AgentOutputParser

In [25]:
import json
from json import JSONDecodeError
from typing import List, Union

import xml.etree.ElementTree as ET

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
)
from langchain_core.outputs import ChatGeneration, Generation

from langchain.agents.agent import AgentOutputParser


class AnthropicFunctionsAgentOutputParser(AgentOutputParser):

    @staticmethod
    def _extract_function_call(full_text):
        # Encontrar o início e o fim da string XML
        if "function_calls" not in full_text:
            return None

        start = full_text.find("<function_calls>")
        end = full_text.find("</function_calls>") + len("</function_calls>")

        # Extrair a string XML
        xml_string = full_text[start:end]

        # Análise da string XML
        root = ET.fromstring(xml_string)

        # Encontrar o nome da ferramenta
        tool_name = root.find(".//tool_name").text

        # Construir o dicionário de parâmetros
        parameters = {}
        for param in root.findall(".//parameters/*"):
            parameters[param.tag] = param.text

        # Construir o dicionário final com nome e parâmetros
        return {
            "name": tool_name,
            "arguments": parameters
        }

    @property
    def _type(self) -> str:
        return "openai-functions-agent"

    @staticmethod
    def _parse_ai_message(message: BaseMessage) -> Union[AgentAction, AgentFinish]:
        """Parse an AI message."""
        if not isinstance(message, AIMessage):
            raise TypeError(f"Expected an AI message got {type(message)}")

        function_call = AnthropicFunctionsAgentOutputParser._extract_function_call(message.content)

        if function_call and 'Final Answer:' not in message.content:
            function_name = function_call["name"]
            try:
                if len(function_call.keys()) == 0:
                    # OpenAI returns an empty string for functions containing no args
                    _tool_input = {}
                else:
                    # otherwise it returns a json object
                    _tool_input = function_call["arguments"]
            except JSONDecodeError:
                raise OutputParserException(
                    f"Could not parse tool input: {function_call} because "
                    f"the `arguments` is not valid JSON."
                )

            
            tool_input = _tool_input

            content_msg = f"responded: {message.content}\n" if message.content else "\n"
            log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
            return AgentActionMessageLog(
                tool=function_name,
                tool_input=tool_input,
                log=log,
                message_log=[message],
            )
        print("\n")
        output = message.content.split("Final Answer:")[-1].strip()

        return AgentFinish(
            return_values={"output": output}, log=str(message.content)
        )

    def parse_result(
            self, result: List[Generation], *, partial: bool = False
        ) -> Union[AgentAction, AgentFinish]:
            if not isinstance(result[0], ChatGeneration):
                raise ValueError("This output parser only works on ChatGeneration output")
            message = result[0].message
            return self._parse_ai_message(message)


    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        raise ValueError("Can only parse messages")


### ReAct Message

In [26]:
import json
from typing import List, Sequence, Tuple

from langchain_core.agents import AgentAction, AgentActionMessageLog
from langchain_core.messages import AIMessage, BaseMessage, FunctionMessage

def _convert_agent_action_to_messages(agent_action: AgentAction, observation: str) -> List[BaseMessage]:
    """Convert an agent action to a message.

    This code is used to reconstruct the original AI message from the agent action.

    Args:
        agent_action: Agent action to convert.

    Returns:
        AIMessage that corresponds to the original tool invocation.
    """
    if isinstance(agent_action, AgentActionMessageLog):
        return str(list(agent_action.message_log) + [_create_function_message(agent_action, observation)])
    else:
        return agent_action.log

def _create_function_message(agent_action: AgentAction, observation: str) -> FunctionMessage:
    """Convert agent action and observation into a function message.
    Args:
        agent_action: the tool invocation request from the agent
        observation: the result of the tool invocation
    Returns:
        FunctionMessage that corresponds to the original tool invocation
    """
    if not isinstance(observation, str):
        try:
            content = json.dumps(observation, ensure_ascii=False)
        except Exception:
            content = str(observation)
    else:
        content = observation
    return agent_action.tool+":"+content


def format_to_openai_function_messages(intermediate_steps: Sequence[Tuple[AgentAction, str]]) -> List[BaseMessage]:
    """Convert (AgentAction, tool output) tuples into FunctionMessages.

    Args:
        intermediate_steps: Steps the LLM has taken to date, along with observations

    Returns:
        list of messages to send to the LLM for the next prediction
    """
    messages = ""

    for agent_action, observation in intermediate_steps:
        messages.extend(_convert_agent_action_to_messages(agent_action, observation))

    return messages


### Run Agent

In [27]:
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template+ "History: {agent_scratchpad}"),

    ]
)


agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    )
    | prompt
    | llm
    | AnthropicFunctionsAgentOutputParser()
)

chain = prompt | llm.bind(stop_sequences=['Final Answer:'])

tools_string = "".join([format_tool_to_anthropic_function(t) for t in tools])

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input":"token 값은 '151C75D463D0601E490A0B1EA283D841'입니다. account id 532805286864의 2023년 9월에서 10월 자원 사용량은?", "tools_string":f"{tools_string}"})



> Entering new AgentExecutor chain...

Invoking: `get_usage` with `{'start_month': '202309', 'end_month': '202310', 'accountId': '532805286864', 'token': '151C75D463D0601E490A0B1EA283D841'}`
responded:  <function_calls>
<invoke>
<tool_name>get_usage</tool_name>
<parameters>
<start_month>202309</start_month>
<end_month>202310</end_month>
<accountId>532805286864</accountId>
<token>151C75D463D0601E490A0B1EA283D841</token>
</parameters>
</invoke>
</function_calls>

420.4350320561
Invoking: `get_usage` with `{'start_month': '202309', 'end_month': '202310', 'accountId': '532805286864', 'token': '151C75D463D0601E490A0B1EA283D841'}`
responded:  <function_calls>
<invoke>
<tool_name>get_usage</tool_name>
<parameters>
<start_month>202309</start_month>
<end_month>202310</end_month>
<accountId>532805286864</accountId>
<token>151C75D463D0601E490A0B1EA283D841</token>
</parameters>
</invoke>
</function_calls>

420.4350320561
Invoking: `get_usage` with `{'start_month': '202309', 'end_month': '202310'

{'input': "token 값은 '151C75D463D0601E490A0B1EA283D841'입니다. account id 532805286864의 2023년 9월에서 10월 자원 사용량은?",
 'tools_string': '<tool_description>\n<tool_name>get_usage</tool_name>\n<description>\nget_usage(start_month: str, end_month: str, accountId: str, token: str) - calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.\n<parameters>\n</parameters>\n</tool_description>',
 'output': 'account id 532805286864의 2023년 9월에서 10월 자원 사용량은 420.4350320561달러입니다.'}